### Версия Python

In [1]:
!python -V

Python 3.12.6


### Импорт необходимых библиотек

In [2]:
# Подавление предупреждений
import warnings
for warn in [UserWarning, FutureWarning]: warnings.filterwarnings("ignore", category = warn)

import os
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
import jupyterlab as jlab

from dataclasses import dataclass

### Версии необходимых библиотек

In [3]:
packages = [
    "Torch", "NumPy", "Pandas", "JupyterLab",
]

package_objects = [
    torch, np, pd, jlab
]

versions = list(map(lambda obj: obj.__version__, package_objects))

pkgs = {"Библиотека": packages, "Версия": versions}
df_pkgs = pd.DataFrame(data = pkgs)
df_pkgs.index.name = "№"
df_pkgs.index += 1

display(df_pkgs)

path_to_reqs = "."
reqs_name = "requirements.txt"

def get_packages_and_versions():
    """Генерация строк с библиотеками и их версиями в формате: библиотека==версия"""
    
    for package, version in zip(packages, versions):
        yield f"{package.lower()}=={version}\n"

with open(os.path.join(path_to_reqs, reqs_name), "w", encoding = "utf-8") as f:
    f.writelines(get_packages_and_versions())

,Библиотека,Версия
№,,
1,Torch,2.2.2
2,NumPy,1.26.4
3,Pandas,2.2.3
4,JupyterLab,4.2.5


### Пример глубокой нейросети

In [4]:
@dataclass
class DNN(nn.Module):
    input_size: int # Входной слой
    hidden_sizes: list[int] # Список размеров скрытых слоев
    output_size: int # Выходной слой

    # Вызывается сразу после __init__ при использовании dataclass
    def __post_init__(self):
        super(DNN, self).__init__()

        # Создание списка для хранения всех слоев
        self.layers = nn.ModuleList()

        # Добавление первого скрытого слоя
        self.layers.append(nn.Linear(self.input_size, self.hidden_sizes[0]))

        # Добавление остальных скрытых слоев
        for i in range(1, len(self.hidden_sizes)):
            self.layers.append(nn.Linear(self.hidden_sizes[i - 1], self.hidden_sizes[i]))
        
        # Добавление выходного слоя
        self.layers.append(nn.Linear(self.hidden_sizes[-1], self.output_size))

        # Определение функции активации для скрытых слоев
        self.g = nn.ReLU()

    # Прямой проход данных через нейросеть
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # Проход через все скрытые слои с функцией активации
        for layer in self.layers[:-1]:
            x = self.g(layer(x))

        # Проход через выходной слой
        x = self.layers[-1](x)

        return x

# Проверка наличия GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Создание экземпляра модели
model = DNN(input_size = 10, hidden_sizes = [20, 30, 20], output_size = 5).to(device)

# Генерация случайного тензора
input = torch.randn(2, model.input_size).to(device)

# Проход через модель
output = model(input)

# Вывод результирующего тензора после прохождения через модель
print("Результирующий тензор после прохождения через модель:")
print(output)

# Вывод размера результирующего тензора
print("Размер результирующего тензора:")
print(output.size())

Результирующий тензор после прохождения через модель:
tensor([[-0.3182,  0.0094,  0.0363, -0.2141, -0.0267],
        [-0.2654,  0.0179, -0.0132, -0.1938, -0.0111]],
       grad_fn=<AddmmBackward0>)
Размер результирующего тензора:
torch.Size([2, 5])
